In [1]:
import rasterio
from rasterio.enums import Resampling
from pathlib import Path
import numpy as np
from rasterio.crs import CRS

In [2]:
input_dir = Path('discrete')
r_files = list(sorted(input_dir.glob('*.asc')))

# Generate band names automatically
num_bands = len(r_files)
band_names = [f"discrete_{i+1:03d}" for i in range(num_bands)]

In [3]:
%%time
# Open all raster files
srcs = [rasterio.open(file) for file in r_files]

# We will use the metadata of the first raster and adjust it for multiple bands
out_meta = srcs[0].meta.copy()

# Ensure the driver is set to 'GTiff' to support multiple bands
out_meta.update({
    'driver': 'GTiff',
    'count': len(srcs),  # number of bands
    'dtype': 'float32'  # update dtype if necessary
})

# Create the output raster
output_raster_path = 'discrete.tif'
with rasterio.open(output_raster_path, 'w', **out_meta) as dst:
    for idx, src in enumerate(srcs, start=1):
        dst.crs=CRS.from_epsg(2193)
        band_data = src.read(1)  # Reading the first band
        dst.write(band_data, idx)  # Writing data to respective band

        # Set description for each band
        dst.set_band_description(idx, band_names[idx - 1])

CPU times: total: 1.53 s
Wall time: 1.9 s


In [4]:
input_dir = Path('continuous')
r_files = list(sorted(input_dir.glob('*.asc')))
# Generate band names automatically
num_bands = len(r_files)
band_names = [f"continuous_{i+1:03d}" for i in range(num_bands)]

In [5]:
%%time
# Open all raster files
srcs = [rasterio.open(file) for file in r_files]

# We will use the metadata of the first raster and adjust it for multiple bands
out_meta = srcs[0].meta.copy()

# Ensure the driver is set to 'GTiff' and update for multiple bands
out_meta.update({
    'driver': 'GTiff',
    'count': len(srcs),
    'dtype': 'int16'  # Using uint8 as we are classifying into 10 classes
})

# Create the output raster
output_raster_path = 'classified_continuous.tif'
with rasterio.open(output_raster_path, 'w', **out_meta) as dst:
    for idx, src in enumerate(srcs, start=1):
        dst.crs=CRS.from_epsg(2193)
        # Read data from each source
        data = src.read(1)

        # Calculate quantiles from 10% to 100% at 10% intervals
        quantiles = np.percentile(data[data != src.nodata], np.arange(10, 101, 10))

        # Classify data based on quantiles
        classified_data = np.searchsorted(quantiles, data, side='right') 

        # Handle nodata values
        classified_data[data == src.nodata] = -9999  # Assuming 0 is nodata for output

        # Write classified data to the respective band in the destination raster
        dst.write(classified_data.astype('uint8'), idx)

        # Set description for each band
        dst.set_band_description(idx, band_names[idx - 1])

CPU times: total: 7.58 s
Wall time: 7.98 s
